In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
from six.moves import range
import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.platform import app
from delf import delf_config_pb2
from delf import feature_io
from delf import utils
from delf import extractor

import delf as df

import sys
import os.path as path

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)

from utiles import lectura_img
import numpy as np

import pickle
import cv2 as cv
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

https://github.com/tensorflow/models/blob/master/research/delf/delf/python/examples/extract_features.py

## Definición de extractor

In [4]:
# Pace to report extraction log.
_STATUS_CHECK_ITERATIONS = 10

def extractorDELF(image_dir, config_path, output_dir, name_pickle):
    
  # Read list of images.
    print('Reading list of images...')
    image_paths = lectura_img(image_dir)
    num_images = len(image_paths)
    print(f'Done! Found {num_images} images')

  # Parse DelfConfig proto.
    config = delf_config_pb2.DelfConfig()
    with tf.io.gfile.GFile(config_path, 'r') as f:
        text_format.Merge(f.read(), config)

  # Create output directory if necessary.
    if not tf.io.gfile.exists(output_dir):
        tf.io.gfile.makedirs(output_dir)
    
  #Create pickle file
    path_pickle = path.abspath(output_dir+'//'+name_pickle)
    pickle_file = open(path_pickle, 'wb')

  #Create lista de descriptor por imagen
    descriptors = list()
    
  #Crea el extractor
    extractor_fn = extractor.MakeExtractor(config)

    start = time.time()
    for i in range(num_images):
    # Report progress once in a while.
        if i == 0:
            print('Starting to extract DELF features from images...')
        elif i % _STATUS_CHECK_ITERATIONS == 0:
            elapsed = (time.time() - start)
            print(
                f'Processing image {i} out of {num_images}, last '
                f'{_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds'
            )
            start = time.time()
            
      #Carga las imágenes
        im = np.array(utils.RgbLoader(image_paths[i]))

      # Extract features.
        extracted_features = extractor_fn(im)
        
      # Save features.
        nom_img = path.split(image_paths[i])[1]
        extracted_features['local_features']['name_img'] = nom_img
        descriptors.append(extracted_features['local_features'])
    pickle.dump(descriptors, pickle_file)
    print("Termino")
    return descriptors

In [5]:
#Output incluye el nombre del archivo
image_dir = '/home/rick/Proyectos/minIA/images/grupoDELF/'
config_path = 'D:\Archivos\Proyectos\minIA\\notebooks\delf_config_example.pbtxt'
config_path_galaxy = '/home/rick/Proyectos/models/research/delf/delf/python/examples/delf_config_galaxy.pbtxt'
output_dir = '/home/rick/Proyectos/images/descr_DELF/'
name_pickle = 'DELF_galaxy.pickle'
descrip = extractorDELF(image_dir, config_path_galaxy, output_dir, name_pickle)

Reading list of images...
Done! Found 31 images
Starting to extract DELF features from images...
Processing image 10 out of 31, last 10 images took 46.38798213005066 seconds
Processing image 20 out of 31, last 10 images took 2.3418498039245605 seconds
Processing image 30 out of 31, last 10 images took 2.372201919555664 seconds
Termino


### Rutas de búsqueda de imágenes

In [42]:
descr_0 = descrip[9]
print("Nombre la de imagen:",descr_0['name_img'])
print("Número de coordenadas (uno por cada Key Point):", len(descr_0['locations']))
print("Número de tamaños (uno por cada Key Point):",len(descr_0['scales']))
print("Número de descriptores en la imagen (uno por cada Key Point):", len(descr_0['descriptors']))
print("Número de puntuaciones (uno por cada Key Point):",len(descr_0['attention']))

print("\n\nDatos para el KP i")
i=3
print("Coordenadas: ", descr_0['locations'][i])
print("Tamaño: ",descr_0['scales'][i])
print("Descriptor: ", descr_0['descriptors'][i])
print("Puntuacion: ", descr_0['attention'][i])

print("Tamaño min: ",min(descr_0['scales']), "Tamaño max: ", max(descr_0['scales']))

Nombre la de imagen: 100367.jpg
Número de coordenadas (uno por cada Key Point): 24
Número de tamaños (uno por cada Key Point): 24
Número de descriptores en la imagen (uno por cada Key Point): 24
Número de puntuaciones (uno por cada Key Point): 24


Datos para el KP i
Coordenadas:  [ 80. 208.]
Tamaño:  0.5
Descriptor:  [-0.03637617 -0.06618628 -0.07843558 -0.1447306  -0.04646507  0.03143152
 -0.03132346  0.14714745 -0.00482944 -0.12908773  0.03126663  0.19503698
  0.03655569  0.08701941 -0.11849143 -0.01943127 -0.05360663  0.0191771
 -0.18015401  0.07055832  0.02661106  0.11522198  0.12514605  0.02480998
  0.1807517  -0.06653397 -0.12351301 -0.0959925  -0.03570505 -0.0460811
  0.03036549  0.03807633  0.016379    0.07855871 -0.01967176 -0.13714121
 -0.03351973 -0.0200255  -0.09031926  0.00285079  0.06439216 -0.04406908
 -0.05217098 -0.00895576  0.027944   -0.14479221 -0.11643306  0.13982198
 -0.02958345 -0.02325598  0.050595    0.02673974  0.08435667 -0.16061065
 -0.10811599 -0.10888302 

### Creación de puntos de interés

In [18]:

#Recibe lista de keypoints de una imagen
def genKeyPoints( kp_img ):
    keypoints = list()
    for i in range (len(kp_img['locations'])):
        keypoints.append(cv.KeyPoint(kp_img['locations'][i][1], kp_img['locations'][i][0], 5*kp_img['scales'][i]))
    return keypoints

In [41]:
@interact
def show_images_per_descriptors(image = [ i for i in range (len(descrip))]):
    descr = descrip[image]
    imagen = path.abspath(image_dir+'/'+descr['name_img'])
    print(imagen)
    img = cv.imread(imagen)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    keypoints = genKeyPoints(descr) 
    img=cv.drawKeypoints(img,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) #Aqui no quitar
    cv.imwrite(output_dir+'\\'+descr['name_img'],img)

    plt.figure(figsize=(20,8))
    plt.imshow(img)

interactive(children=(Dropdown(description='image', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…